In [10]:
from sentiment_analysis_spanish import sentiment_analysis
import emoji
import pandas as pd
from pymongo import MongoClient
import json

In [11]:
def get_mean_likes_per_post(posts_list):
    n_likes = 0
    for post in posts_list:
        n_likes += post["n_likes"]
    mean_likes_per_post = n_likes/len(posts_list)
    
    return mean_likes_per_post

In [12]:
def generate_sentiment_df(sentiment_dict):
    sentiment_df = pd.Series(sentiment_dict, name="sentiment")
    sentiment_df.index.name = "post_id"
    sentiment_df.reset_index()
    
    return sentiment_df

In [13]:
def generate_opensource_instagram_posts(socialnetworkdb, sentiment_df):
    posts_opensource_list = []
    for post in socialnetworkdb.instagram_posts.find({}, {"_id":0}):
        post["sentiment"] = sentiment_df[post["post_id"]]
        posts_opensource_list.append(post)
        
    return posts_opensource_list

In [14]:
def generate_opensource_twitter_posts(socialnetworkdb, sentiment_df):
    posts_opensource_list = []
    for post in socialnetworkdb.twitter_posts.find({}, {"_id":0}):
        post["sentiment"] = sentiment_df[post["post_id"]]
        posts_opensource_list.append(post)
        
    return posts_opensource_list

In [15]:
def write_json(file_name, posts_opensource_list):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(posts_opensource_list, f, ensure_ascii=False, indent=4)

In [17]:
def get_sentiment_per_post(posts_sentiment_list):
    
    # Creation of emojis arrays in order to map the text when there are emojis in it
    positive_emojis = ["👏", "👍", "🙌", "❤️", "🧡", "💛", "💚", "💙", "💜", "🖤", "🤍", "🤎", "♥️", "☺️", "😊", "🔝", "💯", "🎈"]
    negative_emojis = ["👎"]

    # If the sentiment of the post is negative, but the number of likes is sueperior to the mean likes per post, 
    # then it will be considered as neuter.
    mean_likes_per_post = get_mean_likes_per_post(posts_sentiment_list)
    
    sentiment = sentiment_analysis.SentimentAnalysisSpanish()

    sentiment_dict = {}

    for post in posts_sentiment_list:
        sentiment_result = "neuter"
        n_comments = len(post["comments"])
        sentiment_grade = 0
        mean_sentiment_grade = 0
        for comment in post["comments"]:
            # If the comments contain emojis, it is assumed that the text will express the same sentiment as the emojis do.
            if any(emoji in comment["text"] for emoji in positive_emojis):
                comment["text"] = "perfecto"
            elif any(emoji in comment["text"] for emoji in negative_emojis):
                comment["text"] = "mal"
            sentiment_grade += sentiment.sentiment(comment["text"])

        if n_comments > 0:
            mean_sentiment_grade = sentiment_grade/n_comments
        else:
            mean_sentiment_grade = -1


        if (mean_sentiment_grade > 0.7) or ((mean_sentiment_grade == -1) and (post["n_likes"] > mean_likes_per_post)):
            sentiment_result = "positive"
        elif (mean_sentiment_grade != -1) and (mean_sentiment_grade < 0.2) and (post["n_likes"] > mean_likes_per_post): # When something has a negative feeling, there'll be comments showing it.
            sentiment_result = "neuter"
        elif (mean_sentiment_grade != -1) and (mean_sentiment_grade < 0.2):
            sentiment_result = "negative"
        else:
            sentiment_result = "neuter"

        sentiment_dict[post["post_id"]] = sentiment_result
        
    return sentiment_dict